In [1]:
!pip install segmentation_models_pytorch

     |████████████████████████████████| 106 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 549 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 10.7 MB/s eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=0c50478ac306d31c82e145014615553ce435e2b7ec800235ed7af9a844f01d74
  Stored in directory: /Users/ashwindaswani/Library/Caches/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=4aeb694bc172bd53a5cb97d41e384a8b613420b273815244e4c31548d1e99adc
  Stored in directory: /Users/ashwindaswani/Library/Caches/pip/wheels/d1/3b/4e/2f3015f1ab76f34be28e04c4bcee27e8cabfa70d2eadf8bc3b
Successfully built efficientnet-pytorch pretrainedmodels


In [5]:
import segmentation_models_pytorch as smp
import torch, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes
from torch.utils.data import Dataset
from gluoncv.data import CitySegmentation
from PIL import Image
import numpy as np

model = smp.Unet(classes=1, in_channels=3, encoder_name="resnet34", encoder_weights="imagenet")



encoder = smp.encoders.get_encoder("resnet34", in_channels=3)


# Download cityscapes dataset


# load cityscapes dataset

dataset = Cityscapes(root="./data", split="train", mode="fine", transform=transforms.ToTensor(), target_type="color")

# dataset = Cityscapes(root ="./data/" ,split='train', mode='fine', transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

img = "cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_color.png"
import cv2


img = cv2.imread(img)
x = cv2.resize(img, (256, 256))
x = x.transpose(2, 0, 1)
encoded = encoder(torch.from_numpy(x).unsqueeze(0).float())


# show image
# Image.open("cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_color.png")
    



/Users/ashwindaswani/opt/anaconda3/lib/python3.9/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.13.1` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [3]:
def split_dataset():
    train_dataset = Cityscapes(root="./data", split="train", mode="fine", transform=transforms.ToTensor(), target_type="color")
    val_dataset = Cityscapes(root="./data", split="val", mode="fine", transform=transforms.ToTensor(), target_type="color")
    test_dataset = Cityscapes(root="./data", split="test", mode="fine", transform=transforms.ToTensor(), target_type="color")

In [9]:
train_dataset = Cityscapes(root="./data", split="train", mode="fine", transform=transforms.ToTensor(), target_type="color")
val_dataset = Cityscapes(root="./data", split="val", mode="fine", transform=transforms.ToTensor(), target_type="color")
test_dataset = Cityscapes(root="./data", split="test", mode="fine", transform=transforms.ToTensor(), target_type="color")

len(train_dataset), len(val_dataset), len(test_dataset)

(2975, 500, 1525)

In [2]:
import numpy as np
def create_label_index(dataset_size, percent_labeled=0.05):
    return np.random.choice(dataset_size, int(dataset_size * percent_labeled), replace=False)
    
create_label_index(100, 0.05)

array([81, 12, 21, 37, 70])

In [2]:

# mask = "cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_color.png"
# import cv2

def get_classes(mask):
    
    mask = np.array(mask)
    mask = cv2.resize(mask, (256, 256))
    # mask.shape
    final=np.zeros((8,8,3))
    for i in range(8):
        for j in range(8):
            new_img = mask[i*32:(i+1)*32, j*32:(j+1)*32]
            # find the most frequent color
            final[i][j][0] = np.bincount(new_img[:,:,0].flatten()).argmax()
            final[i][j][1] = np.bincount(new_img[:,:,1].flatten()).argmax()
            final[i][j][2] = np.bincount(new_img[:,:,2].flatten()).argmax()
            
    # cv2.imwrite("final.png", final)
    
    return final

In [144]:
X = (8,8,512)
y = (8,8,3)

In [148]:
img = dataset[1][1]

img = np.array(img)
print(img.shape)

(1024, 2048, 4)


In [3]:
encoded_X =[]
for data in dataset:
    X, y = data
    X = X.reshape(1,3,1024,2048)
    
    encoded = model.encoder(X)
    encoded_X.append(encoded[-1])
    y = get_classes(y)
    # y = y.reshape(1,8,8,3)

: 

: 

In [104]:
# show final image
final_img = Image.fromarray(final.astype('uint8'), 'RGB')

In [107]:
final_img.show()

In [70]:
# forwards pass only on encoder
# model = smp.Unet(classes=1, in_channels=3, encoder_name="resnet34", encoder_weights="imagenet")
out = model.encoder(torch.from_numpy(x).unsqueeze(0).float())

In [74]:
out[1].shape


torch.Size([1, 64, 128, 128])

In [76]:
for i in range(len(encoded)):
    print(encoded[i].shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 64, 64])
torch.Size([1, 128, 32, 32])
torch.Size([1, 256, 16, 16])
torch.Size([1, 512, 8, 8])


In [77]:
out = model.encoder(torch.from_numpy(x).unsqueeze(0).float())
out = out[-1]


In [79]:
out.shape

torch.Size([1, 512, 8, 8])

In [82]:
dataset = "cityscapes/gtCoarse/train/aachen"
import os
data = []
for file in os.listdir(dataset):
    if file.endswith(".png") and file.__contains__("color"):
        data.append(file)


In [16]:
dataset = load_dataset("Chris1/GTA5", split="train")

# Get the first 1000 images
images = dataset['train']['image'][:1000]

# Save the images to a directory
for i, image in enumerate(images):
    with open(f'image_{i}.png', 'wb') as f:
        f.write(image)

KeyboardInterrupt: 

In [83]:
len(data)


174

In [84]:
# json_path ="cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_polygons.json"

import json
json_path ="cityscapes/gtCoarse/train/aachen/aachen_000000_000019_gtCoarse_polygons.json"
target = json.load(open(json_path))

In [85]:
mask

{'imgHeight': 1024,
 'imgWidth': 2048,
 'objects': [{'label': 'traffic sign',
   'polygon': [[104, 265], [47, 354], [161, 363], [142, 315], [104, 276]]},
  {'label': 'traffic sign',
   'polygon': [[112, 390], [64, 476], [174, 466], [128, 414]]},
  {'label': 'terrain',
   'polygon': [[13, 582],
    [13, 754],
    [233, 675],
    [322, 593],
    [320, 580],
    [300, 573],
    [278, 558],
    [247, 549],
    [226, 533],
    [177, 521],
    [151, 524],
    [142, 533],
    [143, 606],
    [136, 615],
    [117, 623],
    [104, 623],
    [96, 611],
    [100, 565],
    [107, 546],
    [65, 519],
    [57, 519],
    [51, 525],
    [40, 574],
    [25, 585]]},
  {'label': 'vegetation',
   'polygon': [[358, 236],
    [341, 249],
    [305, 302],
    [300, 335],
    [306, 342],
    [340, 349],
    [380, 350],
    [412, 362],
    [441, 364],
    [478, 345],
    [520, 341],
    [529, 328],
    [533, 321],
    [530, 308],
    [497, 282],
    [482, 279],
    [442, 263],
    [407, 244]]},
  {'label': 'ca

In [ ]:
def initialize_centroids(feat, batch_size, klist, cosine_classifier):
    lbd = cosine_classifier(feat)
    #lbd = lbd.reshape(batch_size, 64, 30)
    assert lbd.shape == (batch_size, 64, 30) 
    lbd = lbd.reshape(batch_size, 64, 30)
    lbd = lbd.reshape(batch_size*64, 30)
    
    masks = lbd
    embeddings = feat.reshape(batch_size* 64, 512)
    
    unique_labels = torch.unique(lbd, dim=0)
    keys = np.arange(30)
    
    indices = [torch.nonzero(masks[:, i], as_tuple=False) for i in range(masks.shape[1])]

    embeddings_list = [embeddings[indices[i]] for i in range(len(indices))]
  
    embeddings_list = [embeddings_list[i].float() for i in range(len(embeddings_list))]
    mean_embeddings = []
    mean_embeddings = [torch.mean(embeddings_list[i], dim=0) for i in range(len(embeddings_list))]
    # print(mean_embeddings)
    #dictionary_indices = dict(zip(keys, indices))
    dictionary_mean = dict(zip(keys, mean_embeddings))
    
    return dictionary_mean

print(initialize_centroids(feat, 1, 30, cosine_classifier))